In [1]:
from google.colab import drive
import sys
drive.mount('/content/drive')
project_dir = '/content/drive/MyDrive/Colab Notebooks/project/final/'
sys.path.append(project_dir+'pages/')

Mounted at /content/drive


In [2]:
%%writefile "/content/drive/MyDrive/Colab Notebooks/project/final/pages/edit_favorites_page.py"
"""
Edit favorites page for the music recommender app.
Allows users to edit their favorite artists.
"""
import sys
import importlib

# Import modules with reload to avoid circular dependencies
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/final/app')
import layout
importlib.reload(layout)

sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/final/app')
import navigation
importlib.reload(navigation)
from navigation import navigate_to, show_status, get_output

sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/final/app')
import session_state
importlib.reload(session_state)
from session_state import session_state

sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/final/handlers')
import favorites_handler
importlib.reload(favorites_handler)
from favorites_handler import FavoritesHandler

sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/final/services')
import user_manager
importlib.reload(user_manager)
from user_manager import UserManager

sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/final/pages')
import base_page
importlib.reload(base_page)
from base_page import BasePage

import ipywidgets as widgets
from IPython.display import display
import pandas as pd

def render_page():
    """Render the edit favorites page."""
    # Create services
    user_manager_instance = UserManager("/content/drive/MyDrive/Colab Notebooks/project/final/data/users.csv")

    # Create handlers
    favorites_handler_instance = FavoritesHandler(user_manager_instance)

    # Create page
    page = EditFavoritesPage(favorites_handler_instance)
    page.render()

class EditFavoritesPage(BasePage):
    """Edit favorites page implementation."""

    def __init__(self, favorites_handler):
        """
        Initialize the edit favorites page.

        Args:
            favorites_handler: Instance of FavoritesHandler
        """
        super().__init__(title="🎨 Edit Favorite Artists")
        self.favorites_handler = favorites_handler
        self.selected_artists = []
        self.artist_input = None
        self.artist_box = None

        # Load artist list
        try:
            songs_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/project/final/data/songs.csv")
            self.artist_list = sorted(songs_df['artist'].dropna().unique().tolist())
        except:
            self.artist_list = []

        # Load current favorites
        self.selected_artists = self.favorites_handler.get_favorite_artists()

    def render(self):
        """Render the edit favorites page."""
        # Add title
        self.add_title()

        # Create form fields
        form_style = {'description_width': '100px'}
        self.artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=self.artist_list,
            description='Artist:',
            ensure_option=True,
            style=form_style
        )

        add_button = widgets.Button(
            description='Add',
            layout=widgets.Layout(width='80px')
        )

        # Artist list container
        self.artist_box = widgets.VBox(layout=widgets.Layout(
            width='100%',
            max_width='400px',
            margin='10px auto'
        ))

        # Add artist handler
        def on_add(b):
            val = self.artist_input.value.strip()
            if val in self.artist_list and val not in self.selected_artists:
                self.selected_artists.append(val)
                self.refresh_artist_table()
                self.artist_input.value = ''
            elif val in self.selected_artists:
                show_status("This artist is already in your favorites", 'warning')
            elif val not in self.artist_list:
                show_status("Artist not found in our database", 'warning')

        add_button.on_click(on_add)

        # Buttons
        save_button = layout.create_button(
            description='Save Changes',
            button_style='primary',
            on_click=lambda b: self.favorites_handler.save_favorite_artists(self.selected_artists)
        )

        cancel_button = layout.create_button(
            description='Cancel',
            on_click=lambda b: navigate_to('session')
        )

        # Artist input container
        artist_input_container = widgets.HBox(
            [self.artist_input, add_button],
            layout=widgets.Layout(width='100%', max_width='400px', margin='0 auto', justify_content='space-between')
        )

        # Header
        artist_header = layout.create_subtitle("Your Current Favorite Artists:")

        # Add widgets to page
        self.add_widget(artist_input_container)
        self.add_widget(artist_header)
        self.add_widget(self.artist_box)
        self.add_spacer(10)

        # Button container
        buttons = layout.create_buttons_container([save_button, cancel_button])
        self.add_widget(buttons)

        # Add output widget
        self.add_widget(get_output())

        # Refresh artist table
        self.refresh_artist_table()

        # Display the page
        self.display()

    def refresh_artist_table(self):
        """Refresh the list of selected artists."""
        rows = []
        for a in self.selected_artists:
            label = widgets.Label(a, layout=widgets.Layout(width='auto', margin='0 10px 0 0'))
            remove_btn = widgets.Button(
                description='Remove',
                button_style='danger',
                layout=widgets.Layout(width='80px')
            )

            def make_remove_callback(artist):
                def remove_callback(b):
                    self.selected_artists.remove(artist)
                    self.refresh_artist_table()
                return remove_callback

            remove_btn.on_click(make_remove_callback(a))

            rows.append(widgets.HBox([
                label, remove_btn
            ], layout=widgets.Layout(justify_content='space-between', width='100%')))

        self.artist_box.children = rows

Overwriting /content/drive/MyDrive/Colab Notebooks/project/final/pages/edit_favorites_page.py
